导入数据

In [1]:
import pandas as pd

data=pd.read_csv('data/上海2019_2023年按月分类文件/20190301_20190401 26184 条.csv')

filename='data/上海2019_2023年按月分类文件_情绪值/20190301_20190401 26184 条_Tencent.csv'

print(len(data)) #数据的行数

data.sample(2) #随机抽样5条数据


26184


,mid,lat,lng,user_name,user_link,verify_typ,weibo_link,publish_time,content,image_urls,video_preview_url,forward_num,comment_num,like_num,phone_type,content_location_name,content_location_url,wgs_lng,wgs_lat
23290,4.355286e+15,31.174976,121.373729,陈起行,https://weibo.com/u/5403378950,没有认证,https://weibo.com/5403378950/Hncegvh1p,2019-03-29 21:00,捷语班新学期首次出游#Alponse Mucha 2上海·上海明珠美术馆,https://wx3.sinaimg.cn/large/005TG1Hogy1g1jypt...,NaN,0.0,1.0,11.0,红米Redmi,上海·上海明珠美术馆,https://weibo.com/p/100101B2094551D369A3FF429C,121.369102,31.176855
11840,4.350541e+15,30.894698,121.091459,青涩天空1015035201,https://weibo.com/u/1015035201,没有认证,https://weibo.com/1015035201/HlcO3rpck,2019-03-16 18:48,分享图片 2上海·金山北站 ​​​​,https://wx3.sinaimg.cn/large/3c803541ly1g14tuh...,NaN,0.0,0.0,1.0,HUAWEI P10 Plus,上海·金山北站,https://weibo.com/p/100101B2094757D069A3FA419F,121.086874,30.896725


腾讯云NLP的情感倾向分析

In [2]:
import json
import types
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.nlp.v20190408 import nlp_client, models
import pandas as pd

key=pd.read_csv('data\key\TencentSecretKey.csv')
SecretId=key['SecretId'][0]
SecretKey=key['SecretKey'][0]

def tencent_nlp(text):
    try:
        # 实例化一个认证对象，入参需要传入腾讯云账户 SecretId 和 SecretKey，此处还需注意密钥对的保密
        # 代码泄露可能会导致 SecretId 和 SecretKey 泄露，并威胁账号下所有资源的安全性。以下代码示例仅供参考，建议采用更安全的方式来使用密钥，请参见：https://cloud.tencent.com/document/product/1278/85305
        # 密钥可前往官网控制台 https://console.cloud.tencent.com/cam/capi 进行获取
        # cred = credential.Credential("SecretId", "SecretKey")
        cred=credential.Credential(SecretId,SecretKey)
        # 实例化一个http选项，可选的，没有特殊需求可以跳过
        httpProfile = HttpProfile()
        httpProfile.endpoint = "nlp.tencentcloudapi.com"

        # 实例化一个client选项，可选的，没有特殊需求可以跳过
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        # 实例化要请求产品的client对象,clientProfile是可选的
        client = nlp_client.NlpClient(cred, "", clientProfile)

        # 实例化一个请求对象,每个接口都会对应一个request对象
        req = models.AnalyzeSentimentRequest()
        params = {
            "Text": text
        }
        req.from_json_string(json.dumps(params))

        # 返回的resp是一个AnalyzeSentimentResponse的实例，与请求对象对应
        resp = client.AnalyzeSentiment(req)
        # 输出json格式的字符串回包
        # print(resp.to_json_string())

        result=json.loads(resp.to_json_string())

        positive_prob=result['Positive']
        neutral_prob=result['Neutral']
        negative_prob=result['Negative']
        sentiment=result['Sentiment']
        # print('正面概率：'+str(positive_prob))
        # print('中性概率：'+str(neutral_prob))
        # print('负面概率：'+str(negative_prob))
        # print('情感倾向：'+str(sentiment))

        return positive_prob,neutral_prob,negative_prob,sentiment

    except TencentCloudSDKException as err:
        print(err)

In [3]:
text="你好, 我是一名学生, 我很开心, 你呢？"

pos,sen,neu,neg=tencent_nlp(text)
print(pos,sen,neu,neg)

0.12495170533657074 0.8017178773880005 0.07333046942949295 neutral


In [ ]:
# 进行数据清洗、情绪值提取#############################################################################

import pandas as pd
import re
import jieba
import pandas as pd
import numpy as np
from time import sleep
import random
import datetime
import csv
import time

def clean_content(content,place):
    # 去除地名
    content = content.replace(place, '')
    # 去除一些关键词
    content = content.replace('分享图片', '').replace('分享视频', '').replace('微博视频', '').replace('的微博视频', '').replace('网页链接','').replace('超话','').replace('新浪图片','').replace('<br>','')    
    # 去除英文
    content = re.sub(r'[a-zA-Z]+', '', content)
    content = re.sub(r'\d+', '', content).replace(' ', '').replace('.', '').replace('_','')
    # 去除所有非中文符号
    content = re.sub(r'[^\u4e00-\u9fa5]', '', content)
    # 去除空白字符
    content = re.sub(r'\s+', '', content)
    return content

time_start=time.time()

# 对content列进行清洗 由对应的地名在文本中去除content文本中的内容
data['clean_content']=data.apply(lambda row: clean_content(row['content'],row['content_location_name']),axis=1)
print("清洗完成"+str(len(data)))

# 如果文本为空 去掉该行
data=data[data['clean_content']!='']
print("去除空白行完成"+str(len(data)))

# 保留文本字数大于3的和小于512的文本   腾讯是200字
data=data[(data['clean_content'].apply(lambda x: len(x)<200)) & (data['clean_content'].apply(lambda x: len(x)>3))]
# data=data[data['clean_content'].apply(lambda x: len(x)>3)]
print("保留字数大于3完成"+str(len(data)))

# 对清洗后的文本 进行 分词 放入jieba_cut列中
data['jieba_cut']=data['clean_content'].apply(lambda x: jieba.lcut(x))
print("分词完成")

def process_row(row):
    try:
        # 如果是nan
        # if pd.isna(row['Tecent_positive']):
            positive_prob, neutral_prob, negative_prob,sentiment=tencent_nlp(row['clean_content'])
            print(datetime.datetime.now(),"一次")
            # 休息几秒钟 确保正确跑完 
            sleep(0.05*random.randint(2,3))
            return positive_prob, neutral_prob, negative_prob,sentiment
        # else:
            # 不做任何操作跳出，下一步
            # return row['Tencent_positive'], row['Tecent_neutral'], row['Tencent_negative'], row['Tencent_sentiment']
    except:
        print(row)
        print(row['clean_content'])
        print("腾讯API调用失败")
        return np.nan,np.nan,np.nan,np.nan
    
results = data.apply(process_row, axis=1)
data['Tencent_positive'], data['Tencent_neutral'], data['Tencent_negative'], data['Tencent_sentiment'] = zip(*results)
print("情感分析完成")

# 保存
# filename='data\WeiboTrainData\sentiment_analysis_data(Baidu)_{}_5.csv'.format(str(datetime.datetime.now()).split(' ')[0])
with open(filename, 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data.columns)  # 写入列名
    for row in data.itertuples(index=False):
        writer.writerow(row)

data.head(5)
print("保存完成")

time_end=time.time()
print(filename,"本次运行总共耗时：",time_end-time_start)
# data.to_csv('data\WeiboTrainData\sentiment_analysis_data(Baidu)_{}_5.csv'.format(str(datetime.datetime.now()).split(' ')[0]), index=False,encoding='utf-8-sig')
# print(data)

In [ ]:
import pandas as pd

###############################################只需要修改filename即可，确定好datainputpath和dataoutputpath这两个文件夹与该py文件的相对位置
filename='20190801_20190901 60340 条.csv'
###############################################注意命名格式是否正确 与csv的名字对应好
datainputpath='data/上海2019_2023年按月分类文件/'
dataoutputpath='data/上海2019_2023年按月分类文件_情绪值/'
data=pd.read_csv(datainputpath+filename)
tailname='_Baidu.csv' # 处理提取sentiment值后的文件

print(len(data)) #数据的行数
data.sample(2) #查看2



# 百度的Key导入 ################################################################################################
import requests
import json

import pandas as pd 
key=pd.read_csv('data\key\BaiduAccesskey.csv')
# key的读取 
API_KEY = key['API_KEY'][0]
SECRET_KEY = key['SECRET_KEY'][0]

def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
    return str(requests.post(url, params=params).json().get("access_token"))

def baidu_nlp(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token=" + get_access_token()
    params = {
        'access_token': 'your_access_token',
        'text': text
    }
    headers = {'Content-Type': 'application/json','Accept': 'application/json'}
        
    response = requests.request("POST",url, data=json.dumps(params), headers=headers)
    
    result = json.loads(response.text)
        
    result=response.json()

    for item in result['items']:
        confidence=item['confidence']
        negative_prob=item['negative_prob']
        positive_prob=item['positive_prob']
        sentiment=item['sentiment']
    
    return confidence, negative_prob, positive_prob,sentiment

# 测试
text="你好, 我是一名学生, 我很开心, 你呢？"
# text="谭鸭血"

c,n,p,s=baidu_nlp(text)
print("confidence",c,"\nnegative_prob",n,"\npositive_prob",p,"\nsentiment",s)


# 进行数据清洗、情绪值提取#############################################################################
import pandas as pd
import re
import jieba
import pandas as pd
import numpy as np
import datetime
from time import sleep
import random
import csv
import time
from tqdm import tqdm

def clean_content(content,place):
    # 去除地名
    content = content.replace(place, '')
    # 去除一些关键词
    content =content.replace('分享图片', '').replace('分享视频', '').replace('微博视频', '').replace('的微博视频', '').replace('网页链接','').replace('超话','').replace('新浪图片','').replace('<br>','')    
    # 去除英文
    content = re.sub(r'[a-zA-Z]+', '', content)
    content = re.sub(r'\d+', '', content).replace(' ', '').replace('.', '').replace('_','')
    # 去除所有非中文符号
    content = re.sub(r'[^\u4e00-\u9fa5]', '', content)
    # 去除空白字符
    content = re.sub(r'\s+', '', content)
    return content

time_start=time.time()

# 对content列进行清洗 由对应的地名在文本中去除content文本中的内容
data['clean_content']=data.apply(lambda row: clean_content(row['content'],row['content_location_name']),axis=1)
print("清洗完成"+str(len(data)))

# 如果文本为空 去掉该行
data=data[data['clean_content']!='']
print("去除空白行完成"+str(len(data)))

# 保留文本字数大于3的和小于512的文本   百度是512字、腾讯是200字
data=data[(data['clean_content'].apply(lambda x: len(x)<512)) & (data['clean_content'].apply(lambda x: len(x)>3))]
print("保留字数大于3完成"+str(len(data)))

# 对清洗后的文本 进行 分词 放入jieba_cut列中
data['jieba_cut']=data['clean_content'].apply(lambda x: jieba.lcut(x))
print("分词完成"+str(len(data)))

def process_row(row):
    try:
        # 如果是nan 这里被标注的if是后续的处理 不用管
        # if pd.isna(row['baidu_confidence']):
            # positive_prob, sentiment, neutral_prob, negative_prob=baidu_nlp(row['clean_content'])
            # print(datetime.datetime.now(),"一次")
            # 休息几秒钟 确保正确跑完 
            sleep(0.04*random.randint(2,3)) # 0.04秒到0.06秒 之间随机休息比较好
            # return positive_prob, sentiment, neutral_prob, negative_prob
        # else:
            # return row['baidu_confidence'], row['baidu_negative'], row['baidu_positive'], row['baidu_sentiment']
    except:
        print(row)
        print(row['clean_content'])
        print("百度API调用失败")
        return np.nan,np.nan,np.nan,np.nan


tqdm.pandas()
# results = data.apply(process_row, axis=1)
results = data.progress_apply(process_row, axis=1)
try: 
    data['baidu_confidence'], data['baidu_negative'], data['baidu_positive'], data['baidu_sentiment'] = zip(*results)
except:
    print("百度API调用失败，将results保存后退出")
    results.to_csv(filename,index=False,encoding='utf-8-sig')
print("情感分析完成")

# 保存
# with open(filename+tailname, 'w', encoding='utf-8-sig', newline='') as file:
with open(dataoutputpath+filename.split('.')[0]+tailname, 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data.columns)  # 写入列名
    for row in data.itertuples(index=False):
        writer.writerow(row)

print(data.head(5))
time_end=time.time()
print(filename,"本次运行总共耗时：",time_end-time_start)
# data.to_csv('data\WeiboTrainData\sentiment_analysis_data(Baidu)_{}_5.csv'.format(str(datetime.datetime.now()).split(' ')[0]), index=False,encoding='utf-8-sig')
# print(data)